In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import autokeras as ak
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Model
from tensorflow.python.keras.utils.np_utils import to_categorical
from sklearn import metrics
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import load_model

# Train the source domain model

In [6]:

for i in np.arange(0,31,1):
    data = pd.read_csv(r"H:\五类数据37\数据2\NP-snv.csv")
    label = pd.read_csv(r'H:\五类数据37\数据2\标签NP.csv')
    enc = OneHotEncoder().fit_transform(label)
    enc.toarray()
    data.columns = np.double(data.columns)
    label = enc.toarray()
    x = np.array(data)
    y = np.array(label)
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.7,random_state = 300)
    y_train = np.array(y_train)
    clf =ak.StructuredDataClassifier(overwrite = True,loss="categorical_crossentropy",metrics="accuracy",max_trials=50)
    clf.fit(x_train,y_train,epochs=300)
    model=clf.export_model()
    tmp = "model_"+str(i)+".h5"
    model.save("tmp")
    loaded_model = load_model(tmp, custom_objects=ak.CUSTOM_OBJECTS)
    loaded_model.summary()
    y_train_predicted = loaded_model.predict(x_train)
    y_test_predicted = loaded_model.predict(x_test)
    for i in range(len(y_train_predicted)):
            max_value=max(y_train_predicted[i])
            for j in range(len(y_train_predicted[i])):
                if max_value==y_train_predicted[i][j]:
                    y_train_predicted[i][j]=1
                else:
                    y_train_predicted[i][j]=0
    for i in range(len(y_test_predicted)):
            max_value=max(y_test_predicted[i])
            for j in range(len(y_test_predicted[i])):
                if max_value==y_test_predicted[i][j]:
                    y_test_predicted[i][j]=1
                else:
                    y_test_predicted[i][j]=0
    from sklearn.metrics import confusion_matrix
    print("\nClassification report for classifier(train):\n\n%s\n"
    % (metrics.classification_report(y_train, y_train_predicted,digits=4)))
    print("Confusion matrix:\n\n%s" % metrics.confusion_matrix(y_train.argmax(axis=1), y_train_predicted.argmax(axis=1)))
    print("\nClassification report for classifier(test):\n\n%s\n"
    % (metrics.classification_report(y_test, y_test_predicted,digits=4)))
    print("Confusion matrix:\n\n%s" % metrics.confusion_matrix(y_test.argmax(axis=1), y_test_predicted.argmax(axis=1)))

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 228)]             0         
                                                                 
 multi_category_encoding (Mu  (None, 228)              0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 228)              457       
 n)                                                              
                                                                 
 dense (Dense)               (None, 256)               58624     
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                             

# Select the source domain model

In [7]:
for i in np.arange(1,11,1):
    data = pd.read_csv(r"H:\五类数据37\数据2\NP-snv.csv")
    label = pd.read_csv(r'H:\五类数据37\数据2\标签NP.csv')
    enc = OneHotEncoder().fit_transform(label)
    enc.toarray()
    data.columns = np.double(data.columns)
    label = enc.toarray()
    x = np.array(data)
    y = np.array(label)
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.7,random_state = 300)
    y_train = np.array(y_train)
    tmp = "model_"+str(i)+".h5"
    loaded_model = load_model(tmp, custom_objects=ak.CUSTOM_OBJECTS)
    loaded_model.summary()
    y_train_predicted = loaded_model.predict(x_train)
    y_test_predicted = loaded_model.predict(x_test)
    for i in range(len(y_train_predicted)):
            max_value=max(y_train_predicted[i])
            for j in range(len(y_train_predicted[i])):
                if max_value==y_train_predicted[i][j]:
                    y_train_predicted[i][j]=1
                else:
                    y_train_predicted[i][j]=0
    for i in range(len(y_test_predicted)):
            max_value=max(y_test_predicted[i])
            for j in range(len(y_test_predicted[i])):
                if max_value==y_test_predicted[i][j]:
                    y_test_predicted[i][j]=1
                else:
                    y_test_predicted[i][j]=0
    from sklearn.metrics import confusion_matrix
    print("\nClassification report for classifier(train):\n\n%s\n"
    % (metrics.classification_report(y_train, y_train_predicted,digits=4)))
    print("Confusion matrix:\n\n%s" % metrics.confusion_matrix(y_train.argmax(axis=1), y_train_predicted.argmax(axis=1)))
    print("\nClassification report for classifier(test):\n\n%s\n"
    % (metrics.classification_report(y_test, y_test_predicted,digits=4)))
    print("Confusion matrix:\n\n%s" % metrics.confusion_matrix(y_test.argmax(axis=1), y_test_predicted.argmax(axis=1)))

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 228)]             0         
                                                                 
 multi_category_encoding (Mu  (None, 228)              0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 228)              457       
 n)                                                              
                                                                 
 dense (Dense)               (None, 128)               29312     
                                                                 
 batch_normalization (BatchN  (None, 128)              512       
 ormalization)                                                   
                                                             

# transfer 3:7

In [2]:
# Import target domain data
x_target_sets = pd.read_csv(r"H:\五类数据37\数据\FM-snv.csv")
y_target_sets = pd.read_csv(r"H:\五类数据37\数据\label_FM.csv")
enc2 = OneHotEncoder().fit_transform(y_target_sets)
enc2.toarray()
x_target_sets.columns = np.double(x_target_sets.columns)
y_target_sets = enc2.toarray()
x_target_sets = np.array(x_target_sets)
x_t_train, x_t_test,y_t_train,y_t_test = train_test_split(x_target_sets,y_target_sets,test_size = 0.7,random_state = 300) 
y_t_train = np.array(y_t_train)

# Import the source domain model

In [3]:
loaded_model = load_model("./model_1.h5", custom_objects=ak.CUSTOM_OBJECTS)
loaded_model.summary()
len(loaded_model.layers)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 228)]             0         
                                                                 
 multi_category_encoding (Mu  (None, 228)              0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 228)              457       
 n)                                                              
                                                                 
 dense (Dense)               (None, 128)               29312     
                                                                 
 batch_normalization (BatchN  (None, 128)              512       
 ormalization)                                                   
                                                             

8

# Take layers from a previously trained model.

In [2]:
from tensorflow.python.keras.layers.core import Dense, Dropout, Activation, Flatten
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras import layers
from tensorflow.keras.layers import Dense

In [6]:
model_new = tf.keras.Sequential()
model_new.add(loaded_model.layers[-8])
model_new.add(loaded_model.layers[-7])
model_new.add(loaded_model.layers[-6])
model_new.add(loaded_model.layers[-5])
model_new.add(loaded_model.layers[-4])
model_new.add(loaded_model.layers[-3])
# model_new.add(loaded_model.layers[-2])
model_new.add(Dense(3,activation = "softmax"))

In [7]:
model_new.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 multi_category_encoding (Mu  (None, 228)              0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 228)              457       
 n)                                                              
                                                                 
 dense (Dense)               (None, 128)               29312     
                                                                 
 batch_normalization (BatchN  (None, 128)              512       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 128)               0         
                                                      

In [ ]:
# 

In [17]:
model_new.trainable = True                                # Freezing layers
model_new.layers[-6].trainable = False
model_new.layers[-5].trainable = False
model_new.layers[-4].trainable = True
model_new.layers[-3].trainable = False
model_new.layers[-2].trainable = True
model_new.layers[-1].trainable = True
model_new.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss = 'categorical_crossentropy',
    metrics="accuracy"
)
for i in np.arange(1,2,1):
    model_new.fit(x_t_train,y_t_train,epochs=500)          #Training new model
inputs = tf.keras.Input(shape=(228,))
model = tf.keras.Model(inputs)
predicted_t_y = model_new.predict(x_t_test)                 #Use the new model to make predictions
model_new.save(r"H:\五类数据37\tran_model\model_new_autokeras1-2.h5")
print(model_new.evaluate(x_t_test, y_t_test))

Epoch 1/500
5/5 [==============================] - 1s 2ms/step - loss: 0.0749 - accuracy: 0.9793
Epoch 2/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0649 - accuracy: 1.0000
Epoch 3/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0596 - accuracy: 0.9931
Epoch 4/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0564 - accuracy: 0.9931
Epoch 5/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0538 - accuracy: 0.9931
Epoch 6/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0520 - accuracy: 1.0000
Epoch 7/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0506 - accuracy: 1.0000
Epoch 8/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0494 - accuracy: 1.0000
Epoch 9/500
5/5 [==============================] - 0s 4ms/step - loss: 0.0486 - accuracy: 1.0000
Epoch 10/500
5/5 [==============================] - 0s 3ms/step - loss: 0.0472 - accuracy: 1.0000
Epoch 11/500
5/5 [===========

In [5]:
x_target_sets = pd.read_csv(r"H:\五类数据37\数据\FM-snv.csv")
y_target_sets = pd.read_csv(r"H:\五类数据37\数据\label_FM.csv")
enc2 = OneHotEncoder().fit_transform(y_target_sets)
enc2.toarray()
x_target_sets.columns = np.double(x_target_sets.columns)
y_target_sets = enc2.toarray()
x_target_sets = np.array(x_target_sets)
x_t_train, x_t_test,y_t_train,y_t_test = train_test_split(x_target_sets,y_target_sets,test_size = 0.7,random_state = 300) 
y_t_train = np.array(y_t_train)
loaded_model = load_model("./model_2.h5", custom_objects=ak.CUSTOM_OBJECTS)
loaded_model.summary()
len(loaded_model.layers)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 228)]             0         
                                                                 
 multi_category_encoding (Mu  (None, 228)              0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 228)              457       
 n)                                                              
                                                                 
 dense (Dense)               (None, 128)               29312     
                                                                 
 batch_normalization (BatchN  (None, 128)              512       
 ormalization)                                                   
                                                             

11

In [20]:
from tensorflow.python.keras.layers.core import Dense, Dropout, Activation, Flatten
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras import layers
from tensorflow.keras.layers import Dense

In [19]:
model_new = tf.keras.Sequential()
model_new.add(loaded_model.layers[-11])
model_new.add(loaded_model.layers[-10])
model_new.add(loaded_model.layers[-9])
model_new.add(loaded_model.layers[-8])
model_new.add(loaded_model.layers[-7])
model_new.add(loaded_model.layers[-6])
model_new.add(loaded_model.layers[-5])
model_new.add(loaded_model.layers[-4])
model_new.add(loaded_model.layers[-3])
# model_new.add(loaded_model.layers[-2])
model_new.add(Dense(3,activation = "softmax"))

In [23]:
model_new.trainable = True
model_new.layers[-9].trainable = False
model_new.layers[-8].trainable = False
model_new.layers[-7].trainable = True
model_new.layers[-6].trainable = False
model_new.layers[-5].trainable = True
model_new.layers[-4].trainable = True
model_new.layers[-3].trainable = False
model_new.layers[-2].trainable = True
model_new.layers[-1].trainable = True
model_new.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001),
    loss = 'categorical_crossentropy',
    metrics="accuracy"
)
for i in np.arange(1,2,1):
    model_new.fit(x_t_train,y_t_train,epochs=500)#训练新模型 
inputs = tf.keras.Input(shape=(228,))
model = tf.keras.Model(inputs)
predicted_t_y = model_new.predict(x_t_test)    #使用新模型进行预测
model_new.save(r"H:\五类数据37\tran_model\model_new_autokeras2-3.h5")
print(model_new.evaluate(x_t_test, y_t_test))

Epoch 1/500
5/5 [==============================] - 1s 2ms/step - loss: 0.0131 - accuracy: 1.0000
Epoch 2/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0128 - accuracy: 1.0000
Epoch 3/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0127 - accuracy: 1.0000
Epoch 4/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0125 - accuracy: 1.0000
Epoch 5/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0124 - accuracy: 1.0000
Epoch 6/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0123 - accuracy: 1.0000
Epoch 7/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0122 - accuracy: 1.0000
Epoch 8/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0121 - accuracy: 1.0000
Epoch 9/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0120 - accuracy: 1.0000
Epoch 10/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0120 - accuracy: 1.0000
Epoch 11/500
5/5 [===========

In [7]:
x_target_sets = pd.read_csv(r"H:\五类数据37\数据\FM-snv.csv")
y_target_sets = pd.read_csv(r"H:\五类数据37\数据\label_FM.csv")
enc2 = OneHotEncoder().fit_transform(y_target_sets)
enc2.toarray()
x_target_sets.columns = np.double(x_target_sets.columns)
y_target_sets = enc2.toarray()
x_target_sets = np.array(x_target_sets)
x_t_train, x_t_test,y_t_train,y_t_test = train_test_split(x_target_sets,y_target_sets,test_size = 0.7,random_state = 300) 
y_t_train = np.array(y_t_train)
loaded_model = load_model("./model_3.h5", custom_objects=ak.CUSTOM_OBJECTS)
loaded_model.summary()
len(loaded_model.layers)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 228)]             0         
                                                                 
 multi_category_encoding (Mu  (None, 228)              0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 228)              457       
 n)                                                              
                                                                 
 dense (Dense)               (None, 32)                7328      
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 1024)              33792 

10

In [25]:
from tensorflow.python.keras.layers.core import Dense, Dropout, Activation, Flatten
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras import layers
from tensorflow.keras.layers import Dense

In [26]:
model_new = tf.keras.Sequential()
model_new.add(loaded_model.layers[-10])
model_new.add(loaded_model.layers[-9])
model_new.add(loaded_model.layers[-8])
model_new.add(loaded_model.layers[-7])
model_new.add(loaded_model.layers[-6])
model_new.add(loaded_model.layers[-5])
model_new.add(loaded_model.layers[-4])
model_new.add(loaded_model.layers[-3])
# model_new.add(loaded_model.layers[-2])
model_new.add(Dense(3,activation = "softmax"))

In [29]:
model_new.trainable = True
model_new.layers[-8].trainable = False
model_new.layers[-7].trainable = False
model_new.layers[-6].trainable = True
model_new.layers[-5].trainable = True
model_new.layers[-4].trainable = True
model_new.layers[-3].trainable = True
model_new.layers[-2].trainable = True
model_new.layers[-1].trainable = True
model_new.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001),
    loss = 'categorical_crossentropy',
    metrics="accuracy"
)
for i in np.arange(1,2,1):
    model_new.fit(x_t_train,y_t_train,epochs=500)#训练新模型 
inputs = tf.keras.Input(shape=(228,))
model = tf.keras.Model(inputs)
predicted_t_y = model_new.predict(x_t_test)    #使用新模型进行预测
model_new.save(r"H:\五类数据37\tran_model\model_new_autokeras3-3.h5")
print(model_new.evaluate(x_t_test, y_t_test))

Epoch 1/500
5/5 [==============================] - 1s 3ms/step - loss: 0.3598 - accuracy: 0.8138
Epoch 2/500
5/5 [==============================] - 0s 3ms/step - loss: 0.3440 - accuracy: 0.8483
Epoch 3/500
5/5 [==============================] - 0s 3ms/step - loss: 0.3699 - accuracy: 0.8138
Epoch 4/500
5/5 [==============================] - 0s 3ms/step - loss: 0.3665 - accuracy: 0.8207
Epoch 5/500
5/5 [==============================] - 0s 2ms/step - loss: 0.3583 - accuracy: 0.8414
Epoch 6/500
5/5 [==============================] - 0s 2ms/step - loss: 0.3612 - accuracy: 0.8345
Epoch 7/500
5/5 [==============================] - 0s 2ms/step - loss: 0.3524 - accuracy: 0.8345
Epoch 8/500
5/5 [==============================] - 0s 2ms/step - loss: 0.3879 - accuracy: 0.8069
Epoch 9/500
5/5 [==============================] - 0s 2ms/step - loss: 0.3504 - accuracy: 0.8276
Epoch 10/500
5/5 [==============================] - 0s 3ms/step - loss: 0.3415 - accuracy: 0.8345
Epoch 11/500
5/5 [===========

In [9]:
x_target_sets = pd.read_csv(r"H:\五类数据37\数据\FM-snv.csv")
y_target_sets = pd.read_csv(r"H:\五类数据37\数据\label_FM.csv")
enc2 = OneHotEncoder().fit_transform(y_target_sets)
enc2.toarray()
x_target_sets.columns = np.double(x_target_sets.columns)
y_target_sets = enc2.toarray()
x_target_sets = np.array(x_target_sets)
x_t_train, x_t_test,y_t_train,y_t_test = train_test_split(x_target_sets,y_target_sets,test_size = 0.7,random_state = 300) 
y_t_train = np.array(y_t_train)
loaded_model = load_model("./model_4.h5", custom_objects=ak.CUSTOM_OBJECTS)
loaded_model.summary()
len(loaded_model.layers)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 228)]             0         
                                                                 
 multi_category_encoding (Mu  (None, 228)              0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 228)              457       
 n)                                                              
                                                                 
 dense (Dense)               (None, 32)                7328      
                                                                 
 batch_normalization (BatchN  (None, 32)               128       
 ormalization)                                                   
                                                             

9

In [31]:
from tensorflow.python.keras.layers.core import Dense, Dropout, Activation, Flatten
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras import layers
from tensorflow.keras.layers import Dense

In [32]:
model_new = tf.keras.Sequential()
model_new.add(loaded_model.layers[-9])
model_new.add(loaded_model.layers[-8])
model_new.add(loaded_model.layers[-7])
model_new.add(loaded_model.layers[-6])
model_new.add(loaded_model.layers[-5])
model_new.add(loaded_model.layers[-4])
model_new.add(loaded_model.layers[-3])
# model_new.add(loaded_model.layers[-2])
model_new.add(Dense(3,activation = "softmax"))

In [41]:
model_new.trainable = True
model_new.layers[-7].trainable = False
model_new.layers[-6].trainable = False
model_new.layers[-5].trainable = True
model_new.layers[-4].trainable = False
model_new.layers[-3].trainable = True
model_new.layers[-2].trainable = True
model_new.layers[-1].trainable = True
model_new.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001),
    loss = 'categorical_crossentropy',
    metrics="accuracy"
)
for i in np.arange(1,2,1):
    history = model_new.fit(x_t_train,y_t_train,epochs=500)#训练新模型 
inputs = tf.keras.Input(shape=(228,))
model = tf.keras.Model(inputs)
predicted_t_y = model_new.predict(x_t_test)    #使用新模型进行预测
model_new.save(r"H:\五类数据37\tran_model\model_new_autokeras4-2.h5")
print(model_new.evaluate(x_t_test, y_t_test))

Epoch 1/500
5/5 [==============================] - 1s 2ms/step - loss: 0.3441 - accuracy: 0.8759
Epoch 2/500
5/5 [==============================] - 0s 2ms/step - loss: 0.3350 - accuracy: 0.8828
Epoch 3/500
5/5 [==============================] - 0s 2ms/step - loss: 0.3431 - accuracy: 0.8897
Epoch 4/500
5/5 [==============================] - 0s 2ms/step - loss: 0.2821 - accuracy: 0.8897
Epoch 5/500
5/5 [==============================] - 0s 2ms/step - loss: 0.3977 - accuracy: 0.8276
Epoch 6/500
5/5 [==============================] - 0s 2ms/step - loss: 0.3743 - accuracy: 0.8828
Epoch 7/500
5/5 [==============================] - 0s 3ms/step - loss: 0.3226 - accuracy: 0.8828
Epoch 8/500
5/5 [==============================] - 0s 2ms/step - loss: 0.3192 - accuracy: 0.8759
Epoch 9/500
5/5 [==============================] - 0s 2ms/step - loss: 0.3127 - accuracy: 0.8897
Epoch 10/500
5/5 [==============================] - 0s 2ms/step - loss: 0.3040 - accuracy: 0.8828
Epoch 11/500
5/5 [===========

In [11]:
x_target_sets = pd.read_csv(r"H:\五类数据37\数据\FM-snv.csv")
y_target_sets = pd.read_csv(r"H:\五类数据37\数据\label_FM.csv")
enc2 = OneHotEncoder().fit_transform(y_target_sets)
enc2.toarray()
x_target_sets.columns = np.double(x_target_sets.columns)
y_target_sets = enc2.toarray()
x_target_sets = np.array(x_target_sets)
x_t_train, x_t_test,y_t_train,y_t_test = train_test_split(x_target_sets,y_target_sets,test_size = 0.7,random_state = 300) 
y_t_train = np.array(y_t_train)
loaded_model = load_model("./model_5.h5", custom_objects=ak.CUSTOM_OBJECTS)
loaded_model.summary()
len(loaded_model.layers)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 228)]             0         
                                                                 
 multi_category_encoding (Mu  (None, 228)              0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 228)              457       
 n)                                                              
                                                                 
 dense (Dense)               (None, 32)                7328      
                                                                 
 batch_normalization (BatchN  (None, 32)               128       
 ormalization)                                                   
                                                             

12

In [44]:
from tensorflow.python.keras.layers.core import Dense, Dropout, Activation, Flatten
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras import layers
from tensorflow.keras.layers import Dense

In [45]:
model_new = tf.keras.Sequential()
model_new.add(loaded_model.layers[-12])
model_new.add(loaded_model.layers[-11])
model_new.add(loaded_model.layers[-10])
model_new.add(loaded_model.layers[-9])
model_new.add(loaded_model.layers[-8])
model_new.add(loaded_model.layers[-7])
model_new.add(loaded_model.layers[-6])
model_new.add(loaded_model.layers[-5])
model_new.add(loaded_model.layers[-4])
model_new.add(loaded_model.layers[-3])
# model_new.add(loaded_model.layers[-2])
model_new.add(Dense(3,activation = "softmax"))

In [64]:
model_new.trainable = True
model_new.layers[-10].trainable = False
model_new.layers[-9].trainable = False
model_new.layers[-8].trainable = True
model_new.layers[-7].trainable = False
model_new.layers[-6].trainable = True
model_new.layers[-5].trainable = True
model_new.layers[-4].trainable = False
model_new.layers[-3].trainable = True
model_new.layers[-2].trainable = True
model_new.layers[-1].trainable = True
model_new.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001),
    loss = 'categorical_crossentropy',
    metrics="accuracy"
)
for i in np.arange(1,2,1):
    history1 = model_new.fit(x_t_train,y_t_train,epochs=500)#训练新模型 
inputs = tf.keras.Input(shape=(228,))
model = tf.keras.Model(inputs)
predicted_t_y = model_new.predict(x_t_test)    #使用新模型进行预测
model_new.save(r"H:\五类数据37\tran_model\model_new_autokeras5-3.h5")
print(model_new.evaluate(x_t_test, y_t_test))

Epoch 1/500
5/5 [==============================] - 1s 3ms/step - loss: 0.0462 - accuracy: 0.9862
Epoch 2/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0619 - accuracy: 0.9862
Epoch 3/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0464 - accuracy: 0.9862
Epoch 4/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0874 - accuracy: 0.9793
Epoch 5/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0446 - accuracy: 0.9931
Epoch 6/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0628 - accuracy: 0.9862
Epoch 7/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0825 - accuracy: 0.9655
Epoch 8/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0652 - accuracy: 0.9862
Epoch 9/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0638 - accuracy: 0.9862
Epoch 10/500
5/5 [==============================] - 0s 2ms/step - loss: 0.0623 - accuracy: 0.9793
Epoch 11/500
5/5 [===========

In [13]:
x_target_sets = pd.read_csv(r"H:\五类数据37\数据\FM-snv.csv")
y_target_sets = pd.read_csv(r"H:\五类数据37\数据\label_FM.csv")
enc2 = OneHotEncoder().fit_transform(y_target_sets)
enc2.toarray()
x_target_sets.columns = np.double(x_target_sets.columns)
y_target_sets = enc2.toarray()
x_target_sets = np.array(x_target_sets)
x_t_train, x_t_test,y_t_train,y_t_test = train_test_split(x_target_sets,y_target_sets,test_size = 0.7,random_state = 300) 
y_t_train = np.array(y_t_train)
loaded_model = load_model("./model_6.h5", custom_objects=ak.CUSTOM_OBJECTS)
loaded_model.summary()
len(loaded_model.layers)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 228)]             0         
                                                                 
 multi_category_encoding (Mu  (None, 228)              0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 228)              457       
 n)                                                              
                                                                 
 dense (Dense)               (None, 64)                14656     
                                                                 
 batch_normalization (BatchN  (None, 64)               256       
 ormalization)                                                   
                                                             

14

In [66]:
from tensorflow.python.keras.layers.core import Dense, Dropout, Activation, Flatten
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras import layers
from tensorflow.keras.layers import Dense

In [67]:
model_new = tf.keras.Sequential()
model_new.add(loaded_model.layers[-14])
model_new.add(loaded_model.layers[-13])
model_new.add(loaded_model.layers[-12])
model_new.add(loaded_model.layers[-11])
model_new.add(loaded_model.layers[-10])
model_new.add(loaded_model.layers[-9])
model_new.add(loaded_model.layers[-8])
model_new.add(loaded_model.layers[-7])
model_new.add(loaded_model.layers[-6])
model_new.add(loaded_model.layers[-5])
model_new.add(loaded_model.layers[-4])
model_new.add(loaded_model.layers[-3])
# model_new.add(loaded_model.layers[-2])
model_new.add(Dense(3,activation = "softmax"))

In [71]:
model_new.trainable = True
model_new.layers[-12].trainable = False
model_new.layers[-11].trainable = False
model_new.layers[-10].trainable = True
model_new.layers[-9].trainable = False
model_new.layers[-8].trainable = True
model_new.layers[-7].trainable = True
model_new.layers[-6].trainable = True
model_new.layers[-5].trainable = False
model_new.layers[-4].trainable = True
model_new.layers[-3].trainable = True
model_new.layers[-2].trainable = True
model_new.layers[-1].trainable = True
model_new.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001),
    loss = 'categorical_crossentropy',
    metrics="accuracy"
)
for i in np.arange(1,2,1):
    history1 = model_new.fit(x_t_train,y_t_train,epochs=500)#训练新模型 
inputs = tf.keras.Input(shape=(228,))
model = tf.keras.Model(inputs)
predicted_t_y = model_new.predict(x_t_test)    #使用新模型进行预测
model_new.save(r"H:\五类数据37\tran_model\model_new_autokeras6-3.h5")
print(model_new.evaluate(x_t_test, y_t_test))

Epoch 1/500
5/5 [==============================] - 1s 2ms/step - loss: 0.6341 - accuracy: 0.7241
Epoch 2/500
5/5 [==============================] - 0s 2ms/step - loss: 0.6320 - accuracy: 0.7034
Epoch 3/500
5/5 [==============================] - 0s 2ms/step - loss: 0.6667 - accuracy: 0.7034
Epoch 4/500
5/5 [==============================] - 0s 2ms/step - loss: 0.6459 - accuracy: 0.6966
Epoch 5/500
5/5 [==============================] - 0s 2ms/step - loss: 0.5930 - accuracy: 0.7724
Epoch 6/500
5/5 [==============================] - 0s 2ms/step - loss: 0.6822 - accuracy: 0.6828
Epoch 7/500
5/5 [==============================] - 0s 2ms/step - loss: 0.6912 - accuracy: 0.6828
Epoch 8/500
5/5 [==============================] - 0s 2ms/step - loss: 0.6061 - accuracy: 0.7379
Epoch 9/500
5/5 [==============================] - 0s 2ms/step - loss: 0.6867 - accuracy: 0.6966
Epoch 10/500
5/5 [==============================] - 0s 2ms/step - loss: 0.5679 - accuracy: 0.7586
Epoch 11/500
5/5 [===========

In [15]:
x_target_sets = pd.read_csv(r"H:\五类数据37\数据\FM-snv.csv")
y_target_sets = pd.read_csv(r"H:\五类数据37\数据\label_FM.csv")
enc2 = OneHotEncoder().fit_transform(y_target_sets)
enc2.toarray()
x_target_sets.columns = np.double(x_target_sets.columns)
y_target_sets = enc2.toarray()
x_target_sets = np.array(x_target_sets)
x_t_train, x_t_test,y_t_train,y_t_test = train_test_split(x_target_sets,y_target_sets,test_size = 0.7,random_state = 300) 
y_t_train = np.array(y_t_train)
loaded_model = load_model("./model_7.h5", custom_objects=ak.CUSTOM_OBJECTS)
loaded_model.summary()
len(loaded_model.layers)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 228)]             0         
                                                                 
 multi_category_encoding (Mu  (None, 228)              0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 228)              457       
 n)                                                              
                                                                 
 dense (Dense)               (None, 32)                7328      
                                                                 
 batch_normalization (BatchN  (None, 32)               128       
 ormalization)                                                   
                                                             

17

In [7]:
from tensorflow.python.keras.layers.core import Dense, Dropout, Activation, Flatten
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras import layers
from tensorflow.keras.layers import Dense

In [10]:
model_new = tf.keras.Sequential()
model_new.add(loaded_model.layers[-17])
model_new.add(loaded_model.layers[-16])
model_new.add(loaded_model.layers[-15])
model_new.add(loaded_model.layers[-14])
model_new.add(loaded_model.layers[-13])
model_new.add(loaded_model.layers[-12])
model_new.add(loaded_model.layers[-11])
model_new.add(loaded_model.layers[-10])
model_new.add(loaded_model.layers[-9])
model_new.add(loaded_model.layers[-8])
model_new.add(loaded_model.layers[-7])
model_new.add(loaded_model.layers[-6])
model_new.add(loaded_model.layers[-5])
model_new.add(loaded_model.layers[-4])
model_new.add(loaded_model.layers[-3])
# model_new.add(loaded_model.layers[-2])
model_new.add(Dense(3,activation = "softmax"))

In [17]:
model_new.trainable = True
model_new.layers[-15].trainable = False
model_new.layers[-14].trainable = False
model_new.layers[-13].trainable = True
model_new.layers[-12].trainable = False
model_new.layers[-11].trainable = True
model_new.layers[-10].trainable = True
model_new.layers[-9].trainable = True
model_new.layers[-8].trainable = False
model_new.layers[-7].trainable = True
model_new.layers[-6].trainable = True
model_new.layers[-5].trainable = True
model_new.layers[-4].trainable = False
model_new.layers[-3].trainable = True
model_new.layers[-2].trainable = True
model_new.layers[-1].trainable = True
model_new.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001),
    loss = 'categorical_crossentropy',
    metrics="accuracy"
)
for i in np.arange(1,2,1):
    history1 = model_new.fit(x_t_train,y_t_train,epochs=500)#训练新模型 
inputs = tf.keras.Input(shape=(228,))
model = tf.keras.Model(inputs)
predicted_t_y = model_new.predict(x_t_test)    #使用新模型进行预测
model_new.save(r"H:\五类数据37\tran_model\model_new_autokeras7-4.h5")
print(model_new.evaluate(x_t_test, y_t_test))

Epoch 1/500
5/5 [==============================] - 1s 3ms/step - loss: 0.4410 - accuracy: 0.8000
Epoch 2/500
5/5 [==============================] - 0s 3ms/step - loss: 0.4222 - accuracy: 0.8276
Epoch 3/500
5/5 [==============================] - 0s 3ms/step - loss: 0.4162 - accuracy: 0.7931
Epoch 4/500
5/5 [==============================] - 0s 3ms/step - loss: 0.4047 - accuracy: 0.8207
Epoch 5/500
5/5 [==============================] - 0s 3ms/step - loss: 0.4921 - accuracy: 0.7655
Epoch 6/500
5/5 [==============================] - 0s 3ms/step - loss: 0.4530 - accuracy: 0.7931
Epoch 7/500
5/5 [==============================] - 0s 3ms/step - loss: 0.4556 - accuracy: 0.8414
Epoch 8/500
5/5 [==============================] - 0s 3ms/step - loss: 0.4022 - accuracy: 0.8138
Epoch 9/500
5/5 [==============================] - 0s 3ms/step - loss: 0.4165 - accuracy: 0.8483
Epoch 10/500
5/5 [==============================] - 0s 3ms/step - loss: 0.4548 - accuracy: 0.8069
Epoch 11/500
5/5 [===========

In [17]:
x_target_sets = pd.read_csv(r"H:\五类数据37\数据\FM-snv.csv")
y_target_sets = pd.read_csv(r"H:\五类数据37\数据\label_FM.csv")
enc2 = OneHotEncoder().fit_transform(y_target_sets)
enc2.toarray()
x_target_sets.columns = np.double(x_target_sets.columns)
y_target_sets = enc2.toarray()
x_target_sets = np.array(x_target_sets)
x_t_train, x_t_test,y_t_train,y_t_test = train_test_split(x_target_sets,y_target_sets,test_size = 0.7,random_state = 300) 
y_t_train = np.array(y_t_train)
loaded_model = load_model("./model_8.h5", custom_objects=ak.CUSTOM_OBJECTS)
loaded_model.summary()
len(loaded_model.layers)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 228)]             0         
                                                                 
 multi_category_encoding (Mu  (None, 228)              0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 228)              457       
 n)                                                              
                                                                 
 dense (Dense)               (None, 32)                7328      
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                                 
 dropout (Dropout)           (None, 32)                0     

11

In [19]:
from tensorflow.python.keras.layers.core import Dense, Dropout, Activation, Flatten
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras import layers
from tensorflow.keras.layers import Dense

In [20]:
model_new = tf.keras.Sequential()
model_new.add(loaded_model.layers[-11])
model_new.add(loaded_model.layers[-10])
model_new.add(loaded_model.layers[-9])
model_new.add(loaded_model.layers[-8])
model_new.add(loaded_model.layers[-7])
model_new.add(loaded_model.layers[-6])
model_new.add(loaded_model.layers[-5])
model_new.add(loaded_model.layers[-4])
model_new.add(loaded_model.layers[-3])
# model_new.add(loaded_model.layers[-2])
model_new.add(Dense(3,activation = "softmax"))

In [25]:
model_new.trainable = True
model_new.layers[-9].trainable = False
model_new.layers[-8].trainable = False
model_new.layers[-7].trainable = True
model_new.layers[-6].trainable = True
model_new.layers[-5].trainable = True
model_new.layers[-4].trainable = True
model_new.layers[-3].trainable = True
model_new.layers[-2].trainable = True
model_new.layers[-1].trainable = True
model_new.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001),
    loss = 'categorical_crossentropy',
    metrics="accuracy"
)
for i in np.arange(1,2,1):
    history1 = model_new.fit(x_t_train,y_t_train,epochs=500)#训练新模型 
inputs = tf.keras.Input(shape=(228,))
model = tf.keras.Model(inputs)
predicted_t_y = model_new.predict(x_t_test)    #使用新模型进行预测
model_new.save(r"H:\五类数据37\tran_model\model_new_autokeras8-3.h5")
print(model_new.evaluate(x_t_test, y_t_test))

Epoch 1/500
5/5 [==============================] - 1s 2ms/step - loss: 0.5163 - accuracy: 0.7793
Epoch 2/500
5/5 [==============================] - 0s 2ms/step - loss: 0.5295 - accuracy: 0.7655
Epoch 3/500
5/5 [==============================] - 0s 2ms/step - loss: 0.5246 - accuracy: 0.7793
Epoch 4/500
5/5 [==============================] - 0s 2ms/step - loss: 0.5014 - accuracy: 0.7931
Epoch 5/500
5/5 [==============================] - 0s 2ms/step - loss: 0.5357 - accuracy: 0.7724
Epoch 6/500
5/5 [==============================] - 0s 2ms/step - loss: 0.6250 - accuracy: 0.7379
Epoch 7/500
5/5 [==============================] - 0s 2ms/step - loss: 0.5179 - accuracy: 0.7724
Epoch 8/500
5/5 [==============================] - 0s 2ms/step - loss: 0.5899 - accuracy: 0.7172
Epoch 9/500
5/5 [==============================] - 0s 2ms/step - loss: 0.6094 - accuracy: 0.7586
Epoch 10/500
5/5 [==============================] - 0s 3ms/step - loss: 0.5434 - accuracy: 0.7586
Epoch 11/500
5/5 [===========

In [19]:
x_target_sets = pd.read_csv(r"H:\五类数据37\数据\FM-snv.csv")
y_target_sets = pd.read_csv(r"H:\五类数据37\数据\label_FM.csv")
enc2 = OneHotEncoder().fit_transform(y_target_sets)
enc2.toarray()
x_target_sets.columns = np.double(x_target_sets.columns)
y_target_sets = enc2.toarray()
x_target_sets = np.array(x_target_sets)
x_t_train, x_t_test,y_t_train,y_t_test = train_test_split(x_target_sets,y_target_sets,test_size = 0.7,random_state = 300) 
y_t_train = np.array(y_t_train)
loaded_model = load_model("./model_9.h5", custom_objects=ak.CUSTOM_OBJECTS)
loaded_model.summary()
len(loaded_model.layers)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 228)]             0         
                                                                 
 multi_category_encoding (Mu  (None, 228)              0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 228)              457       
 n)                                                              
                                                                 
 dense (Dense)               (None, 16)                3664      
                                                                 
 batch_normalization (BatchN  (None, 16)               64        
 ormalization)                                                   
                                                             

13

In [27]:
from tensorflow.python.keras.layers.core import Dense, Dropout, Activation, Flatten
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras import layers
from tensorflow.keras.layers import Dense

In [28]:
model_new = tf.keras.Sequential()
model_new.add(loaded_model.layers[-13])
model_new.add(loaded_model.layers[-12])
model_new.add(loaded_model.layers[-11])
model_new.add(loaded_model.layers[-10])
model_new.add(loaded_model.layers[-9])
model_new.add(loaded_model.layers[-8])
model_new.add(loaded_model.layers[-7])
model_new.add(loaded_model.layers[-6])
model_new.add(loaded_model.layers[-5])
model_new.add(loaded_model.layers[-4])
model_new.add(loaded_model.layers[-3])
# model_new.add(loaded_model.layers[-2])
model_new.add(Dense(3,activation = "softmax"))

In [31]:
model_new.trainable = True
model_new.layers[-11].trainable = False
model_new.layers[-10].trainable = False
model_new.layers[-9].trainable = True
model_new.layers[-8].trainable = False
model_new.layers[-7].trainable = True
model_new.layers[-6].trainable = True
model_new.layers[-5].trainable = True
model_new.layers[-4].trainable = False
model_new.layers[-3].trainable = True
model_new.layers[-2].trainable = True
model_new.layers[-1].trainable = True
model_new.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001),
    loss = 'categorical_crossentropy',
    metrics="accuracy"
)
for i in np.arange(1,2,1):
    history1 = model_new.fit(x_t_train,y_t_train,epochs=500)#训练新模型 
inputs = tf.keras.Input(shape=(228,))
model = tf.keras.Model(inputs)
predicted_t_y = model_new.predict(x_t_test)    #使用新模型进行预测
model_new.save(r"H:\五类数据37\tran_model\model_new_autokeras9-3.h5")
print(model_new.evaluate(x_t_test, y_t_test))

Epoch 1/500
5/5 [==============================] - 1s 2ms/step - loss: 0.6022 - accuracy: 0.7103
Epoch 2/500
5/5 [==============================] - 0s 2ms/step - loss: 0.5571 - accuracy: 0.7517
Epoch 3/500
5/5 [==============================] - 0s 2ms/step - loss: 0.5243 - accuracy: 0.7241
Epoch 4/500
5/5 [==============================] - 0s 2ms/step - loss: 0.5750 - accuracy: 0.7310
Epoch 5/500
5/5 [==============================] - 0s 2ms/step - loss: 0.5449 - accuracy: 0.7862
Epoch 6/500
5/5 [==============================] - 0s 2ms/step - loss: 0.5245 - accuracy: 0.7586
Epoch 7/500
5/5 [==============================] - 0s 2ms/step - loss: 0.5739 - accuracy: 0.7241
Epoch 8/500
5/5 [==============================] - 0s 2ms/step - loss: 0.5394 - accuracy: 0.7655
Epoch 9/500
5/5 [==============================] - 0s 2ms/step - loss: 0.5683 - accuracy: 0.7310
Epoch 10/500
5/5 [==============================] - 0s 2ms/step - loss: 0.5923 - accuracy: 0.7517
Epoch 11/500
5/5 [===========

In [21]:
x_target_sets = pd.read_csv(r"H:\五类数据37\数据\FM-snv.csv")
y_target_sets = pd.read_csv(r"H:\五类数据37\数据\label_FM.csv")
enc2 = OneHotEncoder().fit_transform(y_target_sets)
enc2.toarray()
x_target_sets.columns = np.double(x_target_sets.columns)
y_target_sets = enc2.toarray()
x_target_sets = np.array(x_target_sets)
x_t_train, x_t_test,y_t_train,y_t_test = train_test_split(x_target_sets,y_target_sets,test_size = 0.7,random_state = 300) 
y_t_train = np.array(y_t_train)
loaded_model = load_model("./model_10.h5", custom_objects=ak.CUSTOM_OBJECTS)
loaded_model.summary()
len(loaded_model.layers)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 228)]             0         
                                                                 
 multi_category_encoding (Mu  (None, 228)              0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 228)              457       
 n)                                                              
                                                                 
 dense (Dense)               (None, 32)                7328      
                                                                 
 batch_normalization (BatchN  (None, 32)               128       
 ormalization)                                                   
                                                             

18

In [33]:
from tensorflow.python.keras.layers.core import Dense, Dropout, Activation, Flatten
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras import layers
from tensorflow.keras.layers import Dense

In [34]:
model_new = tf.keras.Sequential()
model_new.add(loaded_model.layers[-18])
model_new.add(loaded_model.layers[-17])
model_new.add(loaded_model.layers[-16])
model_new.add(loaded_model.layers[-15])
model_new.add(loaded_model.layers[-14])
model_new.add(loaded_model.layers[-13])
model_new.add(loaded_model.layers[-12])
model_new.add(loaded_model.layers[-11])
model_new.add(loaded_model.layers[-10])
model_new.add(loaded_model.layers[-9])
model_new.add(loaded_model.layers[-8])
model_new.add(loaded_model.layers[-7])
model_new.add(loaded_model.layers[-6])
model_new.add(loaded_model.layers[-5])
model_new.add(loaded_model.layers[-4])
model_new.add(loaded_model.layers[-3])
# model_new.add(loaded_model.layers[-2])
model_new.add(Dense(3,activation = "softmax"))

In [38]:
model_new.trainable = True
model_new.layers[-16].trainable = False
model_new.layers[-15].trainable = False
model_new.layers[-14].trainable = True
model_new.layers[-13].trainable = False
model_new.layers[-12].trainable = True
model_new.layers[-11].trainable = True
model_new.layers[-10].trainable = True
model_new.layers[-9].trainable = False
model_new.layers[-8].trainable = True
model_new.layers[-7].trainable = True
model_new.layers[-6].trainable = True
model_new.layers[-5].trainable = False
model_new.layers[-4].trainable = True
model_new.layers[-3].trainable = True
model_new.layers[-2].trainable = True
model_new.layers[-1].trainable = True
model_new.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001),
    loss = 'categorical_crossentropy',
    metrics="accuracy"
)
for i in np.arange(1,2,1):
    history1 = model_new.fit(x_t_train,y_t_train,epochs=500)#训练新模型 
inputs = tf.keras.Input(shape=(228,))
model = tf.keras.Model(inputs)
predicted_t_y = model_new.predict(x_t_test)    #使用新模型进行预测
model_new.save(r"H:\五类数据37\tran_model\model_new_autokeras10-4.h5")
print(model_new.evaluate(x_t_test, y_t_test))

Epoch 1/500
5/5 [==============================] - 1s 5ms/step - loss: 0.2922 - accuracy: 0.8828
Epoch 2/500
5/5 [==============================] - 0s 5ms/step - loss: 0.3194 - accuracy: 0.8621
Epoch 3/500
5/5 [==============================] - 0s 5ms/step - loss: 0.2770 - accuracy: 0.8897
Epoch 4/500
5/5 [==============================] - 0s 5ms/step - loss: 0.2847 - accuracy: 0.8690
Epoch 5/500
5/5 [==============================] - 0s 5ms/step - loss: 0.2198 - accuracy: 0.9241
Epoch 6/500
5/5 [==============================] - 0s 5ms/step - loss: 0.3261 - accuracy: 0.8828
Epoch 7/500
5/5 [==============================] - 0s 5ms/step - loss: 0.2871 - accuracy: 0.8759
Epoch 8/500
5/5 [==============================] - 0s 4ms/step - loss: 0.2740 - accuracy: 0.8621
Epoch 9/500
5/5 [==============================] - 0s 4ms/step - loss: 0.2766 - accuracy: 0.8828
Epoch 10/500
5/5 [==============================] - 0s 4ms/step - loss: 0.1968 - accuracy: 0.9241
Epoch 11/500
5/5 [===========

# Viewing model metrics

In [43]:
loaded_model = load_model(r"H:\五类数据37\tran_model\model_new_autokeras1-1.h5", custom_objects=ak.CUSTOM_OBJECTS)
loaded_model.summary()
y_train_predicted = loaded_model.predict(x_t_train)
y_test_predicted = loaded_model.predict(x_t_test)
for i in range(len(y_train_predicted)):
        max_value=max(y_train_predicted[i])
        for j in range(len(y_train_predicted[i])):
            if max_value==y_train_predicted[i][j]:
                y_train_predicted[i][j]=1
            else:
                y_train_predicted[i][j]=0
for i in range(len(y_test_predicted)):
        max_value=max(y_test_predicted[i])
        for j in range(len(y_test_predicted[i])):
            if max_value==y_test_predicted[i][j]:
                y_test_predicted[i][j]=1
            else:
                y_test_predicted[i][j]=0
from sklearn.metrics import confusion_matrix
print("\nClassification report for classifier(train):\n\n%s\n"
% (metrics.classification_report(y_t_train, y_train_predicted,digits=4)))
print("Confusion matrix:\n\n%s" % metrics.confusion_matrix(y_t_train.argmax(axis=1), y_train_predicted.argmax(axis=1)))
print("\nClassification report for classifier(test):\n\n%s\n"
% (metrics.classification_report(y_t_test, y_test_predicted,digits=4)))
print("Confusion matrix:\n\n%s" % metrics.confusion_matrix(y_t_test.argmax(axis=1), y_test_predicted.argmax(axis=1)))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 multi_category_encoding (Mu  (None, 228)              0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 228)              457       
 n)                                                              
                                                                 
 dense (Dense)               (None, 128)               29312     
                                                                 
 batch_normalization (BatchN  (None, 128)              512       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 128)               0         
                                                      

In [44]:
loaded_model = load_model(r"H:\五类数据37\tran_model\model_new_autokeras2-1.h5", custom_objects=ak.CUSTOM_OBJECTS)
loaded_model.summary()
y_train_predicted = loaded_model.predict(x_t_train)
y_test_predicted = loaded_model.predict(x_t_test)
for i in range(len(y_train_predicted)):
        max_value=max(y_train_predicted[i])
        for j in range(len(y_train_predicted[i])):
            if max_value==y_train_predicted[i][j]:
                y_train_predicted[i][j]=1
            else:
                y_train_predicted[i][j]=0
for i in range(len(y_test_predicted)):
        max_value=max(y_test_predicted[i])
        for j in range(len(y_test_predicted[i])):
            if max_value==y_test_predicted[i][j]:
                y_test_predicted[i][j]=1
            else:
                y_test_predicted[i][j]=0
from sklearn.metrics import confusion_matrix
print("\nClassification report for classifier(train):\n\n%s\n"
% (metrics.classification_report(y_t_train, y_train_predicted,digits=4)))
print("Confusion matrix:\n\n%s" % metrics.confusion_matrix(y_t_train.argmax(axis=1), y_train_predicted.argmax(axis=1)))
print("\nClassification report for classifier(test):\n\n%s\n"
% (metrics.classification_report(y_t_test, y_test_predicted,digits=4)))
print("Confusion matrix:\n\n%s" % metrics.confusion_matrix(y_t_test.argmax(axis=1), y_test_predicted.argmax(axis=1)))

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 multi_category_encoding (Mu  (None, 228)              0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 228)              457       
 n)                                                              
                                                                 
 dense (Dense)               (None, 128)               29312     
                                                                 
 batch_normalization (BatchN  (None, 128)              512       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 128)               0         
                                                      

In [45]:
loaded_model = load_model(r"H:\五类数据37\tran_model\model_new_autokeras3-3.h5", custom_objects=ak.CUSTOM_OBJECTS)
loaded_model.summary()
y_train_predicted = loaded_model.predict(x_t_train)
y_test_predicted = loaded_model.predict(x_t_test)
for i in range(len(y_train_predicted)):
        max_value=max(y_train_predicted[i])
        for j in range(len(y_train_predicted[i])):
            if max_value==y_train_predicted[i][j]:
                y_train_predicted[i][j]=1
            else:
                y_train_predicted[i][j]=0
for i in range(len(y_test_predicted)):
        max_value=max(y_test_predicted[i])
        for j in range(len(y_test_predicted[i])):
            if max_value==y_test_predicted[i][j]:
                y_test_predicted[i][j]=1
            else:
                y_test_predicted[i][j]=0
from sklearn.metrics import confusion_matrix
print("\nClassification report for classifier(train):\n\n%s\n"
% (metrics.classification_report(y_t_train, y_train_predicted,digits=4)))
print("Confusion matrix:\n\n%s" % metrics.confusion_matrix(y_t_train.argmax(axis=1), y_train_predicted.argmax(axis=1)))
print("\nClassification report for classifier(test):\n\n%s\n"
% (metrics.classification_report(y_t_test, y_test_predicted,digits=4)))
print("Confusion matrix:\n\n%s" % metrics.confusion_matrix(y_t_test.argmax(axis=1), y_test_predicted.argmax(axis=1)))

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 multi_category_encoding (Mu  (None, 228)              0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 228)              457       
 n)                                                              
                                                                 
 dense (Dense)               (None, 32)                7328      
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 1024)              33792     
                                                                 
 re_lu_1 (ReLU)              (None, 1024)             

In [46]:
loaded_model = load_model(r"H:\五类数据37\tran_model\model_new_autokeras4-2.h5", custom_objects=ak.CUSTOM_OBJECTS)
loaded_model.summary()
y_train_predicted = loaded_model.predict(x_t_train)
y_test_predicted = loaded_model.predict(x_t_test)
for i in range(len(y_train_predicted)):
        max_value=max(y_train_predicted[i])
        for j in range(len(y_train_predicted[i])):
            if max_value==y_train_predicted[i][j]:
                y_train_predicted[i][j]=1
            else:
                y_train_predicted[i][j]=0
for i in range(len(y_test_predicted)):
        max_value=max(y_test_predicted[i])
        for j in range(len(y_test_predicted[i])):
            if max_value==y_test_predicted[i][j]:
                y_test_predicted[i][j]=1
            else:
                y_test_predicted[i][j]=0
from sklearn.metrics import confusion_matrix
print("\nClassification report for classifier(train):\n\n%s\n"
% (metrics.classification_report(y_t_train, y_train_predicted,digits=4)))
print("Confusion matrix:\n\n%s" % metrics.confusion_matrix(y_t_train.argmax(axis=1), y_train_predicted.argmax(axis=1)))
print("\nClassification report for classifier(test):\n\n%s\n"
% (metrics.classification_report(y_t_test, y_test_predicted,digits=4)))
print("Confusion matrix:\n\n%s" % metrics.confusion_matrix(y_t_test.argmax(axis=1), y_test_predicted.argmax(axis=1)))

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 multi_category_encoding (Mu  (None, 228)              0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 228)              457       
 n)                                                              
                                                                 
 dense (Dense)               (None, 32)                7328      
                                                                 
 batch_normalization (BatchN  (None, 32)               128       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                      

In [47]:
loaded_model = load_model(r"H:\五类数据37\tran_model\model_new_autokeras5-1.h5", custom_objects=ak.CUSTOM_OBJECTS)
loaded_model.summary()
y_train_predicted = loaded_model.predict(x_t_train)
y_test_predicted = loaded_model.predict(x_t_test)
for i in range(len(y_train_predicted)):
        max_value=max(y_train_predicted[i])
        for j in range(len(y_train_predicted[i])):
            if max_value==y_train_predicted[i][j]:
                y_train_predicted[i][j]=1
            else:
                y_train_predicted[i][j]=0
for i in range(len(y_test_predicted)):
        max_value=max(y_test_predicted[i])
        for j in range(len(y_test_predicted[i])):
            if max_value==y_test_predicted[i][j]:
                y_test_predicted[i][j]=1
            else:
                y_test_predicted[i][j]=0
from sklearn.metrics import confusion_matrix
print("\nClassification report for classifier(train):\n\n%s\n"
% (metrics.classification_report(y_t_train, y_train_predicted,digits=4)))
print("Confusion matrix:\n\n%s" % metrics.confusion_matrix(y_t_train.argmax(axis=1), y_train_predicted.argmax(axis=1)))
print("\nClassification report for classifier(test):\n\n%s\n"
% (metrics.classification_report(y_t_test, y_test_predicted,digits=4)))
print("Confusion matrix:\n\n%s" % metrics.confusion_matrix(y_t_test.argmax(axis=1), y_test_predicted.argmax(axis=1)))

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 multi_category_encoding (Mu  (None, 228)              0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 228)              457       
 n)                                                              
                                                                 
 dense (Dense)               (None, 32)                7328      
                                                                 
 batch_normalization (BatchN  (None, 32)               128       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                      

In [48]:
loaded_model = load_model(r"H:\五类数据37\tran_model\model_new_autokeras6-3.h5", custom_objects=ak.CUSTOM_OBJECTS)
loaded_model.summary()
y_train_predicted = loaded_model.predict(x_t_train)
y_test_predicted = loaded_model.predict(x_t_test)
for i in range(len(y_train_predicted)):
        max_value=max(y_train_predicted[i])
        for j in range(len(y_train_predicted[i])):
            if max_value==y_train_predicted[i][j]:
                y_train_predicted[i][j]=1
            else:
                y_train_predicted[i][j]=0
for i in range(len(y_test_predicted)):
        max_value=max(y_test_predicted[i])
        for j in range(len(y_test_predicted[i])):
            if max_value==y_test_predicted[i][j]:
                y_test_predicted[i][j]=1
            else:
                y_test_predicted[i][j]=0
from sklearn.metrics import confusion_matrix
print("\nClassification report for classifier(train):\n\n%s\n"
% (metrics.classification_report(y_t_train, y_train_predicted,digits=4)))
print("Confusion matrix:\n\n%s" % metrics.confusion_matrix(y_t_train.argmax(axis=1), y_train_predicted.argmax(axis=1)))
print("\nClassification report for classifier(test):\n\n%s\n"
% (metrics.classification_report(y_t_test, y_test_predicted,digits=4)))
print("Confusion matrix:\n\n%s" % metrics.confusion_matrix(y_t_test.argmax(axis=1), y_test_predicted.argmax(axis=1)))

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 multi_category_encoding (Mu  (None, 228)              0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 228)              457       
 n)                                                              
                                                                 
 dense (Dense)               (None, 64)                14656     
                                                                 
 batch_normalization (BatchN  (None, 64)               256       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 64)                0         
                                                      

In [49]:
loaded_model = load_model(r"H:\五类数据37\tran_model\model_new_autokeras7-4.h5", custom_objects=ak.CUSTOM_OBJECTS)
loaded_model.summary()
y_train_predicted = loaded_model.predict(x_t_train)
y_test_predicted = loaded_model.predict(x_t_test)
for i in range(len(y_train_predicted)):
        max_value=max(y_train_predicted[i])
        for j in range(len(y_train_predicted[i])):
            if max_value==y_train_predicted[i][j]:
                y_train_predicted[i][j]=1
            else:
                y_train_predicted[i][j]=0
for i in range(len(y_test_predicted)):
        max_value=max(y_test_predicted[i])
        for j in range(len(y_test_predicted[i])):
            if max_value==y_test_predicted[i][j]:
                y_test_predicted[i][j]=1
            else:
                y_test_predicted[i][j]=0
from sklearn.metrics import confusion_matrix
print("\nClassification report for classifier(train):\n\n%s\n"
% (metrics.classification_report(y_t_train, y_train_predicted,digits=4)))
print("Confusion matrix:\n\n%s" % metrics.confusion_matrix(y_t_train.argmax(axis=1), y_train_predicted.argmax(axis=1)))
print("\nClassification report for classifier(test):\n\n%s\n"
% (metrics.classification_report(y_t_test, y_test_predicted,digits=4)))
print("Confusion matrix:\n\n%s" % metrics.confusion_matrix(y_t_test.argmax(axis=1), y_test_predicted.argmax(axis=1)))

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 multi_category_encoding (Mu  (None, 228)              0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 228)              457       
 n)                                                              
                                                                 
 dense (Dense)               (None, 32)                7328      
                                                                 
 batch_normalization (BatchN  (None, 32)               128       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                      

In [50]:
loaded_model = load_model(r"H:\五类数据37\tran_model\model_new_autokeras8-3.h5", custom_objects=ak.CUSTOM_OBJECTS)
loaded_model.summary()
y_train_predicted = loaded_model.predict(x_t_train)
y_test_predicted = loaded_model.predict(x_t_test)
for i in range(len(y_train_predicted)):
        max_value=max(y_train_predicted[i])
        for j in range(len(y_train_predicted[i])):
            if max_value==y_train_predicted[i][j]:
                y_train_predicted[i][j]=1
            else:
                y_train_predicted[i][j]=0
for i in range(len(y_test_predicted)):
        max_value=max(y_test_predicted[i])
        for j in range(len(y_test_predicted[i])):
            if max_value==y_test_predicted[i][j]:
                y_test_predicted[i][j]=1
            else:
                y_test_predicted[i][j]=0
from sklearn.metrics import confusion_matrix
print("\nClassification report for classifier(train):\n\n%s\n"
% (metrics.classification_report(y_t_train, y_train_predicted,digits=4)))
print("Confusion matrix:\n\n%s" % metrics.confusion_matrix(y_t_train.argmax(axis=1), y_train_predicted.argmax(axis=1)))
print("\nClassification report for classifier(test):\n\n%s\n"
% (metrics.classification_report(y_t_test, y_test_predicted,digits=4)))
print("Confusion matrix:\n\n%s" % metrics.confusion_matrix(y_t_test.argmax(axis=1), y_test_predicted.argmax(axis=1)))

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 multi_category_encoding (Mu  (None, 228)              0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 228)              457       
 n)                                                              
                                                                 
 dense (Dense)               (None, 32)                7328      
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 256)              

In [51]:
loaded_model = load_model(r"H:\五类数据37\tran_model\model_new_autokeras9-3.h5", custom_objects=ak.CUSTOM_OBJECTS)
loaded_model.summary()
y_train_predicted = loaded_model.predict(x_t_train)
y_test_predicted = loaded_model.predict(x_t_test)
for i in range(len(y_train_predicted)):
        max_value=max(y_train_predicted[i])
        for j in range(len(y_train_predicted[i])):
            if max_value==y_train_predicted[i][j]:
                y_train_predicted[i][j]=1
            else:
                y_train_predicted[i][j]=0
for i in range(len(y_test_predicted)):
        max_value=max(y_test_predicted[i])
        for j in range(len(y_test_predicted[i])):
            if max_value==y_test_predicted[i][j]:
                y_test_predicted[i][j]=1
            else:
                y_test_predicted[i][j]=0
from sklearn.metrics import confusion_matrix
print("\nClassification report for classifier(train):\n\n%s\n"
% (metrics.classification_report(y_t_train, y_train_predicted,digits=4)))
print("Confusion matrix:\n\n%s" % metrics.confusion_matrix(y_t_train.argmax(axis=1), y_train_predicted.argmax(axis=1)))
print("\nClassification report for classifier(test):\n\n%s\n"
% (metrics.classification_report(y_t_test, y_test_predicted,digits=4)))
print("Confusion matrix:\n\n%s" % metrics.confusion_matrix(y_t_test.argmax(axis=1), y_test_predicted.argmax(axis=1)))

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 multi_category_encoding (Mu  (None, 228)              0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 228)              457       
 n)                                                              
                                                                 
 dense (Dense)               (None, 16)                3664      
                                                                 
 batch_normalization (BatchN  (None, 16)               64        
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 16)                0         
                                                      

In [52]:
loaded_model = load_model(r"H:\五类数据37\tran_model\model_new_autokeras10-1.h5", custom_objects=ak.CUSTOM_OBJECTS)
loaded_model.summary()
y_train_predicted = loaded_model.predict(x_t_train)
y_test_predicted = loaded_model.predict(x_t_test)
for i in range(len(y_train_predicted)):
        max_value=max(y_train_predicted[i])
        for j in range(len(y_train_predicted[i])):
            if max_value==y_train_predicted[i][j]:
                y_train_predicted[i][j]=1
            else:
                y_train_predicted[i][j]=0
for i in range(len(y_test_predicted)):
        max_value=max(y_test_predicted[i])
        for j in range(len(y_test_predicted[i])):
            if max_value==y_test_predicted[i][j]:
                y_test_predicted[i][j]=1
            else:
                y_test_predicted[i][j]=0
from sklearn.metrics import confusion_matrix
print("\nClassification report for classifier(train):\n\n%s\n"
% (metrics.classification_report(y_t_train, y_train_predicted,digits=4)))
print("Confusion matrix:\n\n%s" % metrics.confusion_matrix(y_t_train.argmax(axis=1), y_train_predicted.argmax(axis=1)))
print("\nClassification report for classifier(test):\n\n%s\n"
% (metrics.classification_report(y_t_test, y_test_predicted,digits=4)))
print("Confusion matrix:\n\n%s" % metrics.confusion_matrix(y_t_test.argmax(axis=1), y_test_predicted.argmax(axis=1)))

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 multi_category_encoding (Mu  (None, 228)              0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 228)              457       
 n)                                                              
                                                                 
 dense (Dense)               (None, 32)                7328      
                                                                 
 batch_normalization (BatchN  (None, 32)               128       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                      